In [ ]:
import cv2, os, json, pickle, uuid, time
import numpy as np
from NeuralNetStructure import accuracy, activation, dropout,  layer, loss, nnet, optimizers, pool
from preprocessor import Preprocessor
from NeuralNetStructure.imagetools import ImageTools
import numba 



In [ ]:
DATASET_NAME = "HandSigns"
DATASET_PATH = os.path.join("Memory", DATASET_NAME)
CLASSES_PATH = os.path.join(DATASET_PATH, "Classes")
CLASSES = {}

In [ ]:
prep = Preprocessor("HandSigns", (100, 100))


In [ ]:
X, y, X_test, y_test = prep.preprocess(0.80, "2D")

In [ ]:
X.shape, y.shape

In [ ]:
CLASSES = ["ARM_CROSS", "BACK_HAND", "FIST", "FRONT_HAND", "POINT_LEFT", "POINT_RIGHT", "PRAN_MUDRA"]

In [ ]:
y_hot = np.eye(len(CLASSES))[y]
y_test_hot = np.eye(len(CLASSES))[y_test]

In [ ]:
y_hot

In [ ]:
model = nnet.Model("2D", (100, 100))

In [ ]:
model.add(layer.Layer_Conv(input_channels=3, num_filters=1, filter_dims = (5, 5), stride=(1, 1), weight_regularizer_L2=0, bias_regularizer_L2=0), "CONV")
model.add(activation.Activation_ReLU(), "CONV")
model.add(pool.Pool(pool_dims=(2, 2), stride=(2, 2)), "CONV")
model.add(layer.Layer_Conv(input_channels=1, num_filters=3, filter_dims = (5, 5), stride=(1, 1), weight_regularizer_L2=0, bias_regularizer_L2=0), "CONV")
model.add(activation.Activation_ReLU(), "CONV")
model.add(pool.Pool(pool_dims=(2, 2), stride=(2, 2)), "CONV")
model.add(layer.Layer_Flatten(), "CONV")

In [ ]:
model.conv_dims

In [ ]:

dense_feature_num = int(model.conv_dims[-1][0] * model.conv_dims[-1][1] * model.conv_dims[-1][2])

In [ ]:
dense_feature_num

In [ ]:
model.add(layer.Layer_Dense(dense_feature_num, 64, 
    weight_regularizer_L2=1e-4, bias_regularizer_L2=1e-4), 'DENSE')
model.add(activation.Activation_ReLU(), "DENSE")
model.add(dropout.Layer_Dropout(0.2), 'DENSE')

model.add(layer.Layer_Dense(64, 7, 
weight_regularizer_L2=1e-4, bias_regularizer_L2=1e-4), 'DENSE')

model.add(activation.Activation_Sigmoid(), 'DENSE')

model.set(
    loss=loss.Loss_BinaryCrossentropy(),
    optimizer=optimizers.Optimizer_Adam(learning_rate=0.001, decay=5e-7),
    accuracy=accuracy.Accuracy_Binary()
)

model.finalize()

In [ ]:
model.train(X, y_hot,  epochs=100, batch_size = 64, validation_data=(X_test, y_test_hot))


In [ ]:
model.output_layer_activation.predictions(model.output_layer_activation.output)

In [ ]:
model.save_parameters("Memory\\HandSigns\\Parameters")

In [ ]:
#model.save_parameters("Memory\\HandSigns\\Parameters")
model.save("../VisualCortex/handsigns_classifier.nucleus")

In [ ]:
for CLASS_PATH in os.listdir(CLASSES_PATH):
    for img in os.listdir(os.path.join(CLASSES_PATH, CLASS_PATH)):
        name = img
        IMG_PATH = os.path.join(CLASSES_PATH, CLASS_PATH, name)
        img = cv2.imread(IMG_PATH)
        cv2.resize(img, (100, 100))

In [ ]:
mordecAI = nnet.Model.load("../VisualCortex/handsigns_classifier.nucleus")

In [ ]:
model.predict(np.expand_dims(X[0], 0))

In [ ]:
cap = cv2.VideoCapture(0)
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (250,250)
fontScale              = 0.5
fontColor              = (72,251,0)
thickness              = 1
lineType               = 2

In [ ]:


while True:
    ret, frame = cap.read()

    img = cv2.resize(frame[100:300, 220:420, :], (100, 100))
    img = ImageTools.channelLastToFirst(img)

    output = model.forward(np.expand_dims(img, 0), False)
    prediction = CLASSES[np.argmax(output)] if np.max(output) > 0.99 else "NONE"


    cv2.putText(frame,f'{prediction}', 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)

    command = prediction

    cv2.imshow("test", frame[100:300, 220:420, :])

    if cv2.waitKey(1) == 27:
        cv2.destroyAllWindows()
        break